In [1]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 757.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 53.5 MB/s eta 0:00:00


In [2]:
"""
qlstm_reservoir_rl.py
Self-contained RL with a QLSTM-as-Reservoir policy head on a shaped GridWorld.
(Quantum-inspired)
"""

from typing import Tuple, List, Optional
import random
import time
import math

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# ===============================
# Utilities & global configuration
# ===============================
def set_seed(seed: int = 123):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# ===============================
# Environment: shaped GridWorld
# ===============================
class SimpleGridEnv:
    """
    Minimal deterministic GridWorld:
      - Agent starts top-left (0,0)
      - Goal at bottom-right (size-1, size-1)
      - Actions: 0=up, 1=right, 2=down, 3=left
      - Observation: 2 channels (agent one-hot, goal one-hot), flattened to float32 vector
      - Reward shaping: potential-based (Ng et al., 1999) + step penalty
        r' = r + β (γ Φ(s') - Φ(s)) + step_penalty
        where Φ(s) = -ManhattanDistance(agent, goal)
      - Episode ends when goal reached or max_steps exceeded
    """

    def __init__(
        self,
        size: int = 5,
        max_steps: int = 50,
        seed: Optional[int] = None,
        step_penalty: float = -0.01,
        shaping_coef: float = 0.1,
        gamma: float = 0.99,
    ):
        self.size = int(size)
        self.max_steps = int(max_steps)
        self.rng = np.random.RandomState(seed)
        self.step_penalty = float(step_penalty)
        self.shaping_coef = float(shaping_coef)
        self.gamma = float(gamma)

        self.agent_pos = [0, 0]
        self.goal_pos = [self.size - 1, self.size - 1]
        self.steps = 0
        self._last_phi = 0.0

        self.reset()

    @property
    def observation_shape(self) -> tuple:
        return (2 * self.size * self.size,)

    @property
    def action_space_n(self) -> int:
        return 4

    def _phi(self, pos: List[int]) -> float:
        # Negative manhattan distance (larger is better because less negative)
        return - (abs(pos[0] - self.goal_pos[0]) + abs(pos[1] - self.goal_pos[1]))

    def reset(self) -> np.ndarray:
        self.agent_pos = [0, 0]
        self.goal_pos = [self.size - 1, self.size - 1]
        self.steps = 0
        self._last_phi = self._phi(self.agent_pos)
        return self._render_obs()

    def step(self, action: int):
        # Store previous potential
        prev_phi = self._last_phi

        # Move deterministically within bounds
        if action == 0:       # up
            self.agent_pos[0] = max(0, self.agent_pos[0] - 1)
        elif action == 1:     # right
            self.agent_pos[1] = min(self.size - 1, self.agent_pos[1] + 1)
        elif action == 2:     # down
            self.agent_pos[0] = min(self.size - 1, self.agent_pos[0] + 1)
        elif action == 3:     # left
            self.agent_pos[1] = max(0, self.agent_pos[1] - 1)
        else:
            raise ValueError("Invalid action (expected 0..3)")

        self.steps += 1
        reached_goal = (self.agent_pos == self.goal_pos)
        done = reached_goal or (self.steps >= self.max_steps)

        base_reward = 1.0 if reached_goal else 0.0
        new_phi = self._phi(self.agent_pos)
        shaping = self.shaping_coef * (self.gamma * new_phi - prev_phi)

        # add step penalty unless just reached goal this step
        shaped_reward = base_reward + shaping + (0.0 if reached_goal else self.step_penalty)

        self._last_phi = new_phi
        obs = self._render_obs()
        return obs, float(shaped_reward), bool(done), {}

    def _render_obs(self) -> np.ndarray:
        agent_ch = np.zeros((self.size, self.size), dtype=np.float32)
        goal_ch = np.zeros((self.size, self.size), dtype=np.float32)
        agent_ch[self.agent_pos[0], self.agent_pos[1]] = 1.0
        goal_ch[self.goal_pos[0], self.goal_pos[1]] = 1.0
        obs = np.concatenate([agent_ch.ravel(), goal_ch.ravel()]).astype(np.float32)
        return obs

# ===============================
# QLSTM (classical, quantum-inspired)
# ===============================
class QLSTMCell(nn.Module):
    """
    Classical LSTM-like cell with orthonormal-initialized gating to mimic 'unitary-like' stability.
    This serves as a practical stand-in for a QLSTM reservoir (fast, dependency-free).
    """
    def __init__(self, input_size: int, hidden_size: int, bias: bool = True):
        super().__init__()
        self.input_size = int(input_size)
        self.hidden_size = int(hidden_size)
        self.linear = nn.Linear(self.input_size + self.hidden_size, 4 * self.hidden_size, bias=bias)
        self._orthonormal_init(self.linear.weight)
        if bias:
            nn.init.constant_(self.linear.bias, 0.0)

    @staticmethod
    def _orthonormal_init(w: torch.Tensor, gain: float = 1.0):
        try:
            rows, cols = w.shape
            a = torch.randn(rows, cols)
            q, _ = torch.linalg.qr(a)  # QR-based orthonormal
            # crop or pad columns to match shape
            q = q[:, :cols] if q.shape[1] >= cols else torch.nn.functional.pad(q, (0, cols - q.shape[1]))
            with torch.no_grad():
                w.copy_(gain * q)
        except Exception:
            nn.init.xavier_uniform_(w)

    def forward(self, x: torch.Tensor, h: torch.Tensor, c: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        # x: (B, input), h/c: (B, hidden)
        combined = torch.cat([x, h], dim=-1)
        gates = self.linear(combined)
        i, f, o, g = gates.chunk(4, dim=-1)
        i = torch.sigmoid(i)
        f = torch.sigmoid(f)
        o = torch.sigmoid(o)
        g = torch.tanh(g)
        c_next = f * c + i * g
        h_next = o * torch.tanh(c_next)
        return h_next, c_next

class QLSTM(nn.Module):
    """
    Thin wrapper providing single-step and sequence APIs.
    """
    def __init__(self, input_size: int, hidden_size: int):
        super().__init__()
        self.input_size = int(input_size)
        self.hidden_size = int(hidden_size)
        self.cell = QLSTMCell(self.input_size, self.hidden_size)

    def init_hidden(self, batch_size: int = 1, device: Optional[torch.device] = None):
        device = device or torch.device("cpu")
        h = torch.zeros(batch_size, self.hidden_size, device=device)
        c = torch.zeros(batch_size, self.hidden_size, device=device)
        return h, c

    def step(self, x: torch.Tensor, h: torch.Tensor, c: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        return self.cell(x, h, c)

    def forward_sequence(
        self, x_seq: torch.Tensor, h0: Optional[torch.Tensor] = None, c0: Optional[torch.Tensor] = None
    ) -> Tuple[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:
        # x_seq: (B, T, input)
        if x_seq.dim() != 3:
            raise ValueError("x_seq must be (batch, seq_len, input_size)")
        B, T, _ = x_seq.shape
        device = x_seq.device
        if h0 is None or c0 is None:
            h, c = self.init_hidden(B, device=device)
        else:
            h, c = h0, c0
        outputs = []
        for t in range(T):
            xt = x_seq[:, t, :]
            h, c = self.cell(xt, h, c)
            outputs.append(h)
        outputs = torch.stack(outputs, dim=1)  # (B, T, H)
        return outputs, (h, c)

# ===============================
# Reservoir wrapper (stateful)
# ===============================
class ReservoirQLSTM(nn.Module):
    """
    QLSTM used as a reservoir. Optionally freeze parameters.
    Provides a stateful step() API that keeps (h,c) across time and reset() to clear state.
    """
    def __init__(self, qlstm: QLSTM, freeze: bool = True, device: str = "cpu"):
        super().__init__()
        self.qlstm = qlstm
        self.device = torch.device(device)
        if freeze:
            for p in self.qlstm.parameters():
                p.requires_grad = False
        self.h: Optional[torch.Tensor] = None
        self.c: Optional[torch.Tensor] = None

    @property
    def feature_dim(self) -> int:
        return self.qlstm.hidden_size

    def reset(self, batch_size: int = 1):
        self.h, self.c = self.qlstm.init_hidden(batch_size=batch_size, device=self.device)

    def step(self, x_1xD: torch.Tensor) -> torch.Tensor:
        """
        x_1xD: (1, input_dim) float32 tensor on self.device
        returns: (1, feature_dim) current hidden state (h_t)
        """
        if self.h is None or self.c is None:
            self.reset(batch_size=x_1xD.shape[0])
        h_next, c_next = self.qlstm.step(x_1xD, self.h, self.c)
        # Update internal state
        self.h, self.c = h_next.detach(), c_next.detach()
        return self.h

# ===============================
# Actor-Critic network (heads)
# ===============================
class ActorCritic(nn.Module):
    def __init__(self, input_dim: int, n_actions: int, hidden: int = 128):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden)
        self.pi = nn.Linear(hidden, n_actions)  # logits
        self.v = nn.Linear(hidden, 1)

        # Orthogonal initialization is a good default for A2C/A3C
        nn.init.orthogonal_(self.fc1.weight, gain=math.sqrt(2))
        nn.init.orthogonal_(self.pi.weight, gain=0.01)
        nn.init.orthogonal_(self.v.weight, gain=1.0)
        nn.init.constant_(self.fc1.bias, 0.0)
        nn.init.constant_(self.pi.bias, 0.0)
        nn.init.constant_(self.v.bias, 0.0)

    def forward(self, x: torch.Tensor):
        x = F.relu(self.fc1(x))
        logits = self.pi(x)             # (B, A)
        value = self.v(x).squeeze(-1)   # (B,)
        return logits, value

# ===============================
# On-policy trainer with GAE
# ===============================
class OnPolicyTrainer:
    def __init__(
        self,
        env: SimpleGridEnv,
        reservoir: ReservoirQLSTM,
        policy: ActorCritic,
        device: str = "cpu",
        gamma: float = 0.99,
        gae_lambda: float = 0.95,
        lr: float = 1e-4,
        entropy_coef: float = 0.02,     # will anneal down
        value_coef: float = 0.5,
        max_grad_norm: float = 0.5,
        curriculum: bool = False,
    ):
        self.env = env
        self.reservoir = reservoir
        self.policy = policy
        self.device = torch.device(device)
        self.gamma = float(gamma)
        self.gae_lambda = float(gae_lambda)
        self.entropy_coef_init = float(entropy_coef)
        self.value_coef = float(value_coef)
        self.max_grad_norm = float(max_grad_norm)
        self.curriculum = curriculum

        self.reservoir.to(self.device)
        self.policy.to(self.device)
        self.optimizer = optim.Adam(filter(lambda p: p.requires_grad, self.policy.parameters()), lr=lr)

        # tracking
        self.ep_return_ma = 0.0  # moving average of episodic returns
        self.ma_tau = 0.95

    def train(
        self,
        epochs: int = 400,
        rollout_len: int = 32,
        print_every: int = 10,
        normalize_advantage: bool = True,
        entropy_anneal: bool = True,
    ):
        obs = self.env.reset()
        self.reservoir.reset(batch_size=1)
        episode_return = 0.0
        episodes = 0

        for epoch in range(1, epochs + 1):
            rewards: List[torch.Tensor] = []
            values: List[torch.Tensor] = []
            logps: List[torch.Tensor] = []
            entropies: List[torch.Tensor] = []
            masks: List[torch.Tensor] = []

            # Collect a rollout
            for _ in range(rollout_len):
                obs_t = torch.tensor(obs, dtype=torch.float32, device=self.device).unsqueeze(0)  # (1, D)
                feat = self.reservoir.step(obs_t)  # (1, F)

                logits, value = self.policy(feat)  # logits: (1, A), value: (1,)
                probs = F.softmax(logits, dim=-1)
                dist = torch.distributions.Categorical(probs=probs)
                action = dist.sample()
                logp = dist.log_prob(action)              # (1,)
                entropy = dist.entropy().mean()           # scalar

                next_obs, reward, done, _ = self.env.step(int(action.item()))
                episode_return += reward

                rewards.append(torch.tensor(reward, dtype=torch.float32, device=self.device))
                values.append(value.squeeze(0))           # (1,) -> ()
                logps.append(logp.squeeze(0))             # (1,) -> ()
                entropies.append(entropy)
                masks.append(torch.tensor(0.0 if done else 1.0, dtype=torch.float32, device=self.device))

                obs = next_obs

                if done:
                    # episode ends: reset env and reservoir state
                    obs = self.env.reset()
                    self.reservoir.reset(batch_size=1)
                    episodes += 1
                    # update moving average of returns
                    self.ep_return_ma = self.ma_tau * self.ep_return_ma + (1 - self.ma_tau) * episode_return
                    episode_return = 0.0

            # Bootstrap next value from last obs
            obs_t = torch.tensor(obs, dtype=torch.float32, device=self.device).unsqueeze(0)
            feat = self.reservoir.step(obs_t)  # note: keeps state continuity across rollouts
            with torch.no_grad():
                _, next_value = self.policy(feat)
                next_value = next_value.squeeze(0)

            # GAE-Lambda advantages
            advs: List[torch.Tensor] = []
            gae = torch.tensor(0.0, dtype=torch.float32, device=self.device)
            prev_value = next_value
            for t in reversed(range(len(rewards))):
                delta = rewards[t] + self.gamma * prev_value * masks[t] - values[t]
                gae = delta + self.gamma * self.gae_lambda * masks[t] * gae
                advs.insert(0, gae)
                prev_value = values[t]

            returns = [advs[i] + values[i] for i in range(len(values))]

            # Stack
            returns_t = torch.stack(returns)
            advs_t = torch.stack(advs)
            values_t = torch.stack(values)
            logps_t = torch.stack(logps)
            entropies_t = torch.stack(entropies)

            # Normalize advantages to reduce variance
            if normalize_advantage:
                advs_t = (advs_t - advs_t.mean()) / (advs_t.std(unbiased=False) + 1e-8)

            # Anneal entropy coef from initial -> 0.001 over training
            if entropy_anneal:
                frac = 1.0 - (epoch / max(1, epochs))
                ent_coef = 0.001 + frac * (self.entropy_coef_init - 0.001)
            else:
                ent_coef = self.entropy_coef_init

            # Losses
            policy_loss = - (logps_t * advs_t.detach()).mean()
            value_loss = F.mse_loss(values_t, returns_t.detach())
            entropy_loss = entropies_t.mean()
            loss = policy_loss + self.value_coef * value_loss - ent_coef * entropy_loss

            # Optimize
            self.optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(filter(lambda p: p.requires_grad, self.policy.parameters()), self.max_grad_norm)
            self.optimizer.step()

            # Basic prints
            if epoch % print_every == 0:
                avg_return = returns_t.mean().item() if returns_t.numel() > 0 else 0.0
                print(f"[Epoch {epoch}] "
                      f"loss={loss.item():.4f} policy_loss={policy_loss.item():.4f} "
                      f"value_loss={value_loss.item():.4f} entropy={entropy_loss.item():.4f} "
                      f"avg_return={avg_return:.3f} ent_coef={ent_coef:.4f} "
                      f"episodes={episodes} ep_return_ma={self.ep_return_ma:.3f}")

            # Optional simple curriculum: if learning well, increase grid size
            if self.curriculum and (epoch % 50 == 0) and (self.ep_return_ma > 0.6) and (self.env.size < 7):
                self.env.size += 1
                self.env.max_steps += 10
                # reposition goal to new bottom-right
                self.env.goal_pos = [self.env.size - 1, self.env.size - 1]
                print(f"Curriculum: increased grid to {self.env.size}x{self.env.size}")

# ===============================
# Main
# ===============================
def main():
    set_seed(123)

    # Environment with shaping; tweak coefficients as needed
    env = SimpleGridEnv(
        size=5,
        max_steps=50,
        seed=123,
        step_penalty=-0.01,
        shaping_coef=0.1,
        gamma=0.99,
    )

    obs_dim = env.observation_shape[0]
    n_actions = env.action_space_n

    # Reservoir QLSTM
    qlstm_hidden = 32
    qlstm = QLSTM(input_size=obs_dim, hidden_size=qlstm_hidden)
    reservoir = ReservoirQLSTM(qlstm=qlstm, freeze=True, device="cpu")
    # Actor-Critic heads on top of reservoir features
    reservoir.reset(batch_size=1)
    policy = ActorCritic(input_dim=reservoir.feature_dim, n_actions=n_actions, hidden=128)

    # Trainer
    trainer = OnPolicyTrainer(
        env=env,
        reservoir=reservoir,
        policy=policy,
        device="cpu",
        gamma=0.99,
        gae_lambda=0.95,
        lr=1e-4,
        entropy_coef=0.02,
        value_coef=0.5,
        max_grad_norm=0.5,
        curriculum=False,  # set True to gradually increase grid size
    )

    t0 = time.time()
    trainer.train(
        epochs=400,            # increase for better performance
        rollout_len=32,        # 32-64 works well here
        print_every=20,
        normalize_advantage=True,
        entropy_anneal=True,
    )
    t1 = time.time()
    print(f"Training complete. Elapsed: {t1 - t0:.2f}s")

    # Save learned policy head (reservoir is frozen)
    torch.save(policy.state_dict(), "actor_critic_head.pth")
    print("Saved policy head to actor_critic_head.pth")

if __name__ == "__main__":
    main()


[Epoch 20] loss=0.1063 policy_loss=-0.0016 value_loss=0.2686 entropy=1.3863 avg_return=0.210 ent_coef=0.0191 episodes=14 ep_return_ma=0.405
[Epoch 40] loss=-0.0199 policy_loss=-0.0029 value_loss=0.0162 entropy=1.3863 avg_return=-0.049 ent_coef=0.0181 episodes=28 ep_return_ma=0.478
[Epoch 60] loss=0.1721 policy_loss=-0.0028 value_loss=0.3974 entropy=1.3862 avg_return=0.509 ent_coef=0.0171 episodes=44 ep_return_ma=0.819
[Epoch 80] loss=0.1441 policy_loss=-0.0057 value_loss=0.3445 entropy=1.3861 avg_return=0.245 ent_coef=0.0162 episodes=60 ep_return_ma=0.782
[Epoch 100] loss=-0.0211 policy_loss=-0.0073 value_loss=0.0146 entropy=1.3860 avg_return=0.014 ent_coef=0.0152 episodes=75 ep_return_ma=0.673
[Epoch 120] loss=-0.0074 policy_loss=-0.0070 value_loss=0.0388 entropy=1.3859 avg_return=0.060 ent_coef=0.0143 episodes=90 ep_return_ma=0.770
[Epoch 140] loss=-0.0230 policy_loss=-0.0100 value_loss=0.0110 entropy=1.3856 avg_return=0.139 ent_coef=0.0134 episodes=105 ep_return_ma=0.728
[Epoch 160]

In [19]:
#!/usr/bin/env python3

from typing import Tuple, List, Optional
import argparse
import math
import time
import copy
import random

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import sys

try:
    import pennylane as qml
except Exception:
    qml = None
    print("WARNING: pennylane not available. Set up pennylane if you want quantum circuits.")

# -------------------------
# Hyperparameters (paper-inspired)
# -------------------------
DEFAULT_LR = 1e-4
BETA1 = 0.92
BETA2 = 0.999
GAMMA = 0.9                 # per paper
GAE_LAMBDA = 0.95
ENTROPY_INIT = 0.02
VALUE_COEF = 0.5
MAX_GRAD_NORM = 0.5

# -------------------------
# Utilities
# -------------------------
def set_seed(seed: int = 1234):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# -------------------------
# Small MiniGrid-like environment (no gym)
# Partial observation (local view) to make memory beneficial.
# -------------------------
class MiniGridLite:
    def __init__(self, size: int = 5, max_steps: int = 4 * 5, view_size: int = 3,
                 step_penalty: float = -0.01, shaping_coef: float = 0.1, gamma: float = GAMMA,
                 random_start: bool = False, seed: Optional[int] = None):
        assert view_size % 2 == 1
        self.size = int(size)
        self.max_steps = int(max_steps)
        self.view_size = int(view_size)
        self.step_penalty = float(step_penalty)
        self.shaping_coef = float(shaping_coef)
        self.gamma = float(gamma)
        self.random_start = bool(random_start)
        self.rng = np.random.RandomState(seed)
        self.reset()

    @property
    def observation_shape(self) -> Tuple[int]:
        view_elems = self.view_size * self.view_size * 2
        return (view_elems + 2,)

    @property
    def action_space_n(self) -> int:
        return 4

    def _phi(self, pos: List[int]) -> float:
        return - (abs(pos[0] - self.goal_pos[0]) + abs(pos[1] - self.goal_pos[1]))

    def reset(self) -> np.ndarray:
        if self.random_start:
            self.agent_pos = [int(self.rng.randint(0, self.size)), int(self.rng.randint(0, self.size))]
        else:
            self.agent_pos = [0, 0]
        self.goal_pos = [self.size - 1, self.size - 1]
        self.steps = 0
        self._last_phi = self._phi(self.agent_pos)
        return self._render_obs()

    def step(self, action: int):
        prev_phi = self._last_phi
        if action == 0:
            self.agent_pos[0] = max(0, self.agent_pos[0] - 1)
        elif action == 1:
            self.agent_pos[1] = min(self.size - 1, self.agent_pos[1] + 1)
        elif action == 2:
            self.agent_pos[0] = min(self.size - 1, self.agent_pos[0] + 1)
        elif action == 3:
            self.agent_pos[1] = max(0, self.agent_pos[1] - 1)
        else:
            raise ValueError("action must be 0..3")

        self.steps += 1
        reached = (self.agent_pos == self.goal_pos)
        done = reached or (self.steps >= self.max_steps)
        base_reward = 1.0 if reached else 0.0
        new_phi = self._phi(self.agent_pos)
        shaping = self.shaping_coef * (self.gamma * new_phi - prev_phi)
        r = base_reward + shaping + (0.0 if reached else self.step_penalty)
        self._last_phi = new_phi
        obs = self._render_obs()
        return obs, float(r), bool(done), {}

    def _render_obs(self) -> np.ndarray:
        half = self.view_size // 2
        ax, ay = self.agent_pos
        view = np.zeros((self.view_size, self.view_size, 2), dtype=np.float32)
        for dx in range(-half, half + 1):
            for dy in range(-half, half + 1):
                x = ax + dx; y = ay + dy
                xi = dx + half; yi = dy + half
                if 0 <= x < self.size and 0 <= y < self.size:
                    view[xi, yi, 0] = 1.0
                    if [x, y] == self.goal_pos:
                        view[xi, yi, 1] = 1.0
                        view[xi, yi, 0] = 0.0
                else:
                    view[xi, yi, 0] = 0.0
                    view[xi, yi, 1] = 0.0
        relx = (self.goal_pos[0] - self.agent_pos[0]) / float(max(1, self.size - 1))
        rely = (self.goal_pos[1] - self.agent_pos[1]) / float(max(1, self.size - 1))
        extra = np.array([relx, rely], dtype=np.float32)
        obs = np.concatenate([view.ravel(), extra]).astype(np.float32)
        return obs

# -------------------------
# PennyLane QLSTM cell (VQC)
# - encoding: H + RY(arctan(x)) + RZ(arctan(x^2))
# - chain CNOT entangler
# - Rot(a,b,c) variational per wire per layer
# - measure Z per wire -> classical features -> gate readout
# -------------------------
class PennyQLSTMCell(nn.Module):
    def __init__(self, in_dim: int, hidden_dim: int, n_qubits: int = 4, n_layers: int = 1,
                 device_name: str = "default.qubit", freeze: bool = True):
        super().__init__()
        if qml is None:
            raise RuntimeError("PennyLane required for PennyQLSTMCell. Install pennylane.")

        self.in_dim = int(in_dim)
        self.hidden_dim = int(hidden_dim)
        self.n_qubits = int(n_qubits)
        self.n_layers = int(n_layers)
        self.device_name = device_name
        self.freeze = bool(freeze)

        # classical encoder and readout
        self.encode_lin = nn.Linear(self.in_dim + self.hidden_dim, self.n_qubits)
        nn.init.xavier_uniform_(self.encode_lin.weight)
        nn.init.constant_(self.encode_lin.bias, 0.0)

        self.gate_lin = nn.Linear(self.n_qubits, 4 * self.hidden_dim)
        nn.init.xavier_uniform_(self.gate_lin.weight)
        nn.init.constant_(self.gate_lin.bias, 0.0)

        # quantum params (3 per qubit per layer)
        self.total_qparams = self.n_layers * self.n_qubits * 3
        init_q = 0.05 * torch.randn(self.total_qparams)
        self.q_params = nn.Parameter(init_q, requires_grad=(not self.freeze))

        # lazy qnode
        self._qnode = None

    def build_qnode(self):
        # create process-local device / qnode (here single-process)
        dev = qml.device(self.device_name, wires=self.n_qubits)

        @qml.qnode(dev, interface="torch", diff_method="parameter-shift")
        def qnode(x_enc, qparams):
            # encoding
            for w in range(self.n_qubits):
                qml.H(wires=w)
                qml.RY(torch.atan(x_enc[w]), wires=w)
                qml.RZ(torch.atan(x_enc[w] ** 2), wires=w)
            idx = 0
            for _ in range(self.n_layers):
                for w in range(self.n_qubits - 1):
                    qml.CNOT(wires=[w, w + 1])
                for w in range(self.n_qubits):
                    a = qparams[idx]; b = qparams[idx + 1]; c = qparams[idx + 2]
                    qml.Rot(a, b, c, wires=w)
                    idx += 3
            return [qml.expval(qml.PauliZ(w)) for w in range(self.n_qubits)]

        self._qnode = qnode

    def forward(self, obs_vec: torch.Tensor, h_prev: torch.Tensor, c_prev: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        if self._qnode is None:
            self.build_qnode()

        combined = torch.cat([obs_vec, h_prev], dim=0)
        x_enc = self.encode_lin(combined)  # shape (n_qubits,)

        # execute qnode: prefer torch.Tensor result
        q_out = self._qnode(x_enc, self.q_params)
        q_feats = torch.as_tensor(q_out, dtype=torch.float32)

        # safe conversion and device handling:
        #if isinstance(q_out, torch.Tensor):
        #    q_feats = q_out
       # else:
            # if q_out is list/np, convert to CPU float tensor
       #     q_feats = torch.tensor(q_out, dtype=torch.float32, device='cpu')

        # if reservoir frozen, detach quantum outputs immediately to avoid autograd graphs linking into quantum side
        if self.freeze:
            q_feats = q_feats.detach()

        # move q_feats to same device as obs_vec (likely CPU)
        q_feats = q_feats.to(obs_vec.device)

        gates = self.gate_lin(q_feats)  # (4 * hidden_dim,)
        i, f, o, g = gates.chunk(4, dim=-1)
        i = torch.sigmoid(i); f = torch.sigmoid(f); o = torch.sigmoid(o); g = torch.tanh(g)
        c_next = f * c_prev + i * g
        h_next = o * torch.tanh(c_next)
        return h_next, c_next

# -------------------------
# ActorCritic heads (classical)
# -------------------------
class ActorCritic(nn.Module):
    def __init__(self, input_dim: int, n_actions: int, hidden: int = 128):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden)
        self.pi = nn.Linear(hidden, n_actions)
        self.v = nn.Linear(hidden, 1)
        nn.init.orthogonal_(self.fc1.weight, gain=math.sqrt(2))
        nn.init.constant_(self.fc1.bias, 0.0)
        nn.init.orthogonal_(self.pi.weight, gain=0.01)
        nn.init.constant_(self.pi.bias, 0.0)
        nn.init.orthogonal_(self.v.weight, gain=1.0)
        nn.init.constant_(self.v.bias, 0.0)

    def forward(self, x: torch.Tensor):
        x = F.relu(self.fc1(x))
        logits = self.pi(x)
        value = self.v(x).squeeze(-1)
        return logits, value

# -------------------------
# Hybrid model: classical preproc -> QLSTM cell -> actor/critic (classical)
# -------------------------
class HybridModel(nn.Module):
    def __init__(self, obs_dim: int, n_actions: int, q_hidden: int = 32, q_n_qubits: int = 4, q_n_layers: int = 1,
                 preproc_hidden: int = 64, head_hidden: int = 128, freeze_quantum: bool = True, q_device: str = "default.qubit"):
        super().__init__()
        self.preproc = nn.Sequential(nn.Linear(obs_dim, preproc_hidden), nn.ReLU())
        self.qlstm_cell = PennyQLSTMCell(in_dim=preproc_hidden, hidden_dim=q_hidden, n_qubits=q_n_qubits, n_layers=q_n_layers, device_name=q_device, freeze=freeze_quantum)
        self.actor_head = nn.Sequential(nn.Linear(q_hidden, head_hidden), nn.ReLU(), nn.Linear(head_hidden, n_actions))
        self.critic_head = nn.Sequential(nn.Linear(q_hidden, head_hidden), nn.ReLU(), nn.Linear(head_hidden, 1))

        # init classical linears
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.orthogonal_(m.weight, gain=math.sqrt(2))
                nn.init.constant_(m.bias, 0.0)

    def init_hidden(self, device: Optional[torch.device] = None):
        device = device or torch.device("cpu")
        return torch.zeros(self.qlstm_cell.hidden_dim, device=device), torch.zeros(self.qlstm_cell.hidden_dim, device=device)

    def forward_step(self, obs_np: np.ndarray, h_prev: torch.Tensor, c_prev: torch.Tensor):
        obs = torch.tensor(obs_np, dtype=torch.float32)
        x = self.preproc(obs)
        h_next, c_next = self.qlstm_cell.forward(x, h_prev, c_prev)
        logits = self.actor_head(h_next)
        value = self.critic_head(h_next).squeeze(-1)
        return logits, value, (h_next, c_next)

# -------------------------
# On-policy trainer (single-process)
# - collects rollouts of fixed length, computes GAE, advantage normalization,
#   entropy annealing, and performs single optimizer.step per update.
# -------------------------
class OnPolicyTrainer:
    def __init__(self, env: MiniGridLite, model: HybridModel, device: str = "cpu",
                 lr: float = DEFAULT_LR, gamma: float = GAMMA, gae_lambda: float = GAE_LAMBDA,
                 entropy_coef_init: float = ENTROPY_INIT, value_coef: float = VALUE_COEF,
                 max_grad_norm: float = MAX_GRAD_NORM):
        self.env = env
        self.model = model
        self.device = torch.device(device)
        self.gamma = float(gamma)
        self.gae_lambda = float(gae_lambda)
        self.entropy_coef_init = float(entropy_coef_init)
        self.value_coef = float(value_coef)
        self.max_grad_norm = float(max_grad_norm)

        # optimizer: include quantum params only if they are trainable
        params = [p for p in self.model.parameters() if p.requires_grad]
        if len(params) == 0:
            raise RuntimeError("No trainable params found (maybe quantum reservoir frozen and no classical params?).")
        self.optimizer = optim.Adam(params, lr=lr, betas=(BETA1, BETA2))

        self.ep_return_ma = 0.0
        self.ma_tau = 0.95

    def train(self, epochs: int = 400, rollout_len: int = 32, print_every: int = 10,
              normalize_advantage: bool = True, entropy_anneal: bool = True):
        obs = self.env.reset()
        h, c = self.model.init_hidden()
        episode_return = 0.0
        episodes = 0

        for epoch in range(1, epochs + 1):
            rewards: List[torch.Tensor] = []
            values: List[torch.Tensor] = []
            logps: List[torch.Tensor] = []
            entropies: List[torch.Tensor] = []
            masks: List[torch.Tensor] = []

            # collect rollout
            for t in range(rollout_len):
                logits, value, (h, c) = self.model.forward_step(obs, h, c)
                probs = F.softmax(logits, dim=-1)
                dist = torch.distributions.Categorical(probs=probs)
                action = dist.sample()
                logp = dist.log_prob(action)
                entropy = dist.entropy().mean()

                next_obs, reward, done, _ = self.env.step(int(action.item()))
                episode_return += reward

                rewards.append(torch.tensor(reward, dtype=torch.float32))
                values.append(value)
                logps.append(logp)
                entropies.append(entropy)
                masks.append(torch.tensor(0.0 if done else 1.0, dtype=torch.float32))

                obs = next_obs
                if done:
                    obs = self.env.reset()
                    h, c = self.model.init_hidden()
                    episodes += 1
                    self.ep_return_ma = self.ma_tau * self.ep_return_ma + (1 - self.ma_tau) * episode_return
                    episode_return = 0.0

            # bootstrap from last obs without advancing QLSTM (we used forward_step so state already advanced)
            # We will compute next value using current hidden (h,c) but not advance quantum state
            with torch.no_grad():
                logits, next_value, _ = self.model.forward_step(obs, h, c)
                next_value = next_value

            # compute GAE
            advs: List[torch.Tensor] = []
            gae = 0.0
            prev_value = next_value
            for i in reversed(range(len(rewards))):
                delta = rewards[i].to(self.device) + self.gamma * prev_value * masks[i].to(self.device) - values[i].to(self.device)
                gae = delta + self.gamma * self.gae_lambda * masks[i].to(self.device) * gae
                advs.insert(0, gae)
                prev_value = values[i].to(self.device)

            returns = [advs[i] + values[i].to(self.device) for i in range(len(values))]

            # stack for loss
            returns_t = torch.stack(returns)
            advs_t = torch.stack(advs)
            values_t = torch.stack([v.to(self.device) for v in values])
            logps_t = torch.stack([lp.to(self.device) for lp in logps])
            entropies_t = torch.stack([e.to(self.device) for e in entropies])

            if normalize_advantage:
                advs_t = (advs_t - advs_t.mean()) / (advs_t.std(unbiased=False) + 1e-8)

            # anneal entropy
            if entropy_anneal:
                frac = 1.0 - (epoch / max(1, epochs))
                ent_coef = 0.001 + frac * (self.entropy_coef_init - 0.001)
            else:
                ent_coef = self.entropy_coef_init

            policy_loss = - (logps_t * advs_t.detach()).mean()
            value_loss = F.mse_loss(values_t, returns_t.detach())
            entropy_loss = entropies_t.mean()
            loss = policy_loss + self.value_coef * value_loss - ent_coef * entropy_loss

            # optimize
            self.optimizer.zero_grad()
            loss.backward(retain_graph=True)
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.max_grad_norm)
            self.optimizer.step()

            if epoch % print_every == 0:
                avg_return = returns_t.mean().item() if returns_t.numel() > 0 else 0.0
                print(f"[Epoch {epoch}] loss={loss.item():.4f} policy_loss={policy_loss.item():.4f} value_loss={value_loss.item():.4f} entropy={entropy_loss.item():.4f} avg_return={avg_return:.3f} ep_return_ma={self.ep_return_ma:.3f} ent_coef={ent_coef:.4f}")

# -------------------------
# Main
# -------------------------
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--epochs", type=int, default=400)
    parser.add_argument("--rollout_len", type=int, default=32)
    parser.add_argument("--print_every", type=int, default=10)
    parser.add_argument("--grid_size", type=int, default=5)
    parser.add_argument("--view_size", type=int, default=3)
    parser.add_argument("--q_n_qubits", type=int, default=4)
    parser.add_argument("--q_n_layers", type=int, default=2)
    parser.add_argument("--q_hidden", type=int, default=32)
    parser.add_argument("--preproc_hidden", type=int, default=64)
    parser.add_argument("--head_hidden", type=int, default=128)
    parser.add_argument("--lr", type=float, default=DEFAULT_LR)
    parser.add_argument("--train-quantum", action="store_true", help="Include quantum params in optimization (slow).")
    args, unknown = parser.parse_known_args(sys.argv[1:])

    set_seed(1234)

    # env
    env = MiniGridLite(size=args.grid_size, max_steps=4 * args.grid_size, view_size=args.view_size)

    obs_dim = env.observation_shape[0]
    n_actions = env.action_space_n

    model = HybridModel(obs_dim=obs_dim, n_actions=n_actions,
                        q_hidden=args.q_hidden, q_n_qubits=args.q_n_qubits, q_n_layers=args.q_n_layers,
                        preproc_hidden=args.preproc_hidden, head_hidden=args.head_hidden,
                        freeze_quantum=(not args.train_quantum),
                        q_device=("lightning.qubit" if qml is not None else "default.qubit"))

    trainer = OnPolicyTrainer(env=env, model=model, device="cpu", lr=args.lr,
                              gamma=GAMMA, gae_lambda=GAE_LAMBDA,
                              entropy_coef_init=ENTROPY_INIT, value_coef=VALUE_COEF,
                              max_grad_norm=MAX_GRAD_NORM)

    t0 = time.time()
    trainer.train(epochs=args.epochs, rollout_len=args.rollout_len, print_every=args.print_every)
    t1 = time.time()
    print(f"Training done. Elapsed {t1 - t0:.2f}s")

    torch.save(model.state_dict(), "qlstm_sync_model.pth")
    print("Saved model to qlstm_sync_model.pth")

if __name__ == "__main__":
    main()


[Epoch 10] loss=0.0198 policy_loss=0.0014 value_loss=0.0910 entropy=1.3863 avg_return=0.235 ep_return_ma=0.732 ent_coef=0.0195
[Epoch 20] loss=0.0134 policy_loss=0.0015 value_loss=0.0766 entropy=1.3863 avg_return=0.204 ep_return_ma=1.088 ent_coef=0.0191
[Epoch 30] loss=0.0151 policy_loss=-0.0017 value_loss=0.0851 entropy=1.3862 avg_return=0.230 ep_return_ma=1.191 ent_coef=0.0186
[Epoch 40] loss=0.0257 policy_loss=-0.0036 value_loss=0.1086 entropy=1.3862 avg_return=0.343 ep_return_ma=1.229 ent_coef=0.0181
[Epoch 50] loss=0.0115 policy_loss=-0.0036 value_loss=0.0791 entropy=1.3862 avg_return=0.275 ep_return_ma=1.321 ent_coef=0.0176
[Epoch 60] loss=-0.0009 policy_loss=-0.0066 value_loss=0.0589 entropy=1.3861 avg_return=0.214 ep_return_ma=1.374 ent_coef=0.0171
[Epoch 70] loss=0.0038 policy_loss=-0.0090 value_loss=0.0719 entropy=1.3860 avg_return=0.266 ep_return_ma=1.328 ent_coef=0.0167
[Epoch 80] loss=0.0982 policy_loss=-0.0126 value_loss=0.2665 entropy=1.3860 avg_return=0.525 ep_return_ma